# Določevanje razpoloženja (sentimenta)

## Namestitev programskih modulov

In [ ]:
!pip install transformers
!pip install python-docx
!pip install ipywidgets

## Priklic programskih modulov

In [ ]:
from transformers import pipeline
from docx import Document
import re


## Priklic besedila

In [ ]:
# Naloži dokument
file_path = '/content/Sporocilo sodelavcem univerze o kibernapadu anonimizirano.docx'
doc = Document(file_path)


## Priprava besedila

V prvem koraku pridobimo besedilo iz naložene Wordove datoteke.

V drugem koraku razdelimo besedilo na povedi. Merilo segmentacije so končna ločila. Pri tem opravilu nam pomagajo regularni izrazi.

In [ ]:
# Izvleči besedilo iz dokumenta
text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
sentences = re.split(r'(?<=[.!?]) +', text)


## Priklic sentimentnega modela

In [ ]:
# Naloži model sentimentne analize
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")


## Analiza besedila

In [ ]:
# Analiziraj sentiment posamezne povedi
results = [{"sentence": sentence, "sentiment": sentiment_analysis(sentence)[0]} for sentence in sentences]


## Prikaz izidov

In [ ]:
# Prikaži rezultate
for result in results:
    print(f"Sentence: {result['sentence']}")
    print(f"Sentiment: {result['sentiment']['label']} (Score: {result['sentiment']['score']})\n")


## Sentiment v barvah

S funkcijo get_color() obarvamo povedi besedila obarvamo glede na vrednost sentimenta (positive, neutral, negative).

Nato ustvarimo osnovo za spletno stran v ustreznem oblikovnem slogu (naslov, nabor in velikost črk, velikost strani, barve).

Sledi dodeljevanje izbranih barv (green, gray, red) posameznim povedim glede na izračunano vrednost razpoloženja (sentimenta).

Na koncu shranimo spletno stran v datoteko s pripono 'html'.

Datoteko lahko prenesemo na svoj računalnik in odpremo v brskalniku, npr. če v levem panelu odpremo mapo, izberemo 'Prenos' in v Raziskovalcu kliknemo html datoteko (privzeto jo najdemo v mapi 'Prenosi' / 'Downloads').



In [ ]:
from IPython.core.display import display, HTML

# Map labels to colors
def get_color(label):
    if label == "POSITIVE":
        return "green"
    elif label == "NEGATIVE":
        return "red"
    return "gray"  # Neutral

# Create an HTML string with proper formatting
html = """<!DOCTYPE html>
<html>
<head>
    <title>Sentiment Analysis</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.8; margin: 20px; }
        .positive { color: green; }
        .neutral { color: gray; }
        .negative { color: red; }
    </style>
</head>
<body>
<h1>Sentiment Analysis Results</h1>
<p>Below are the sentences from the document, colored based on their sentiment:</p>
"""

for result in results:
    label = result["sentiment"]["label"].lower()
    html += f"<p class='{label}'>{result['sentence']}</p>\n"

html += "</body></html>"

# Save the HTML to a file
output_file = "sentiment_analysis_colored.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html)

print(f"HTML file saved as {output_file}. Open it in a browser to view the results.")